# Errors and Exception Handling

In [1]:
print('Hello)

SyntaxError: EOL while scanning string literal (<ipython-input-1-db8c9988558c>, line 1)

Note how we get a SyntaxError, with the further description that it was an EOL (End of Line Error) while scanning the string literal. This is specific enough for us to see that we forgot a single quote at the end of the line. 

This type of error and description is known as an Exception. Even if a statement or expression is syntactically correct, it may cause an error when an attempt is made to execute it. Errors detected during execution are called exceptions and are not unconditionally fatal. You can check out the full list of built-in exceptions [here](https://docs.python.org/2/library/exceptions.html).

Also notice the **traceback / stack trace** provided, showing us where the error occured. The error raised and its description is printed after.

Best practices for evaluating an error are:
1. Look at your code.
2. Google the error and look for StackOverflow results.
3. Look at your code slowly, running through it as if you were the computer.
4. Run only some parts of the code in isolation to determine where it is coming from.
5. Use a debugger.

Errors are most helpful for developers.

#### Error Principles

Errors can be used to make it explicit when something didn't work as intended, so that the caller can deal with the failure (asking for forgiveness).

Another option, less popular in Python, is to check whether the thing you're trying to do will succeed before doing it (asking for permission).

## Common Errors

- IndexError -> index doesn't exist
- KeyError -> key doesn't exist
- NameError -> variable is not defined
- AttributeError -> object doesn't have a method/attribute
- NotImplementedError -> raise this if a feature hasn't been implemented yet
- RuntimeErrror -> base class, lots of errors extend this class
- SyntaxError -> any illegal syntax
- IndentationError -> indentation is incorrect
- TabError -> indentation is not consistent (1 tab vs 4 spaces)
- TypeError -> type was not expected for a specific method
- ValueError -> correct type, but incorrect value
- ImportError -> circular imports when 2+ files are all importing each other infinitely
- DecprecationWarning -> there is now a better way of doing what you're trying to do


## Raising Errors

Notice the use of `isinstance(variable, class)` to check if a variable is of a specific class.

In [8]:
class Car:
    def __init__(self, make, model):
        self.make = make
        self.model = model
    
    def __repr__(self):
        return f'<Car {self.make} {self.model}>'


class Garage:
    def __init__(self):
        self.cars = []
    
    def __len__(self):
        return len(self.cars)
    
    def add_car(self, car):
        if not isinstance(car, Car):
            raise TypeError(f'Tried to add {car.__class__.__name__}, but you can only add Car objects.')
        
        raise NotImplementedError('We can\'t add cars to the garage yet.')

            
ford = Garage()
ford.add_car('Fiesta')

TypeError: Tried to add str, but you can only add Car objects.

In [9]:
car = Car('Ford', 'Fiesta')
ford.add_car(car)

NotImplementedError: We can't add cars to the garage yet.

## Creating Errors

Instead of using these errors, it's often better to create your own, more specific errors. It's best practice to check for valid inputs inside your class.

In [10]:
class RuntimeErrorWithCode(Exception):
    """
    Exception raised when a specific error code is needed.
    """
    def __init__(self, message, code):
        super().__init__(f'Error code {code}: {message}')
        self.code = code


err = RuntimeErrorWithCode('An error happened.', 500)
print(err.__doc__)


    Exception raised when a specific error code is needed.
    


## try and except

The basic terminology and syntax used to handle errors in Python is the **try** and **except** statements. The code which can cause an exception to occur is put in the *try* block and the handling of the exception is the implemented in the *except* block of code:

    try:
       You do your operations here...
       ...
    except ExceptionI:
       If there is ExceptionI, then execute this block.
    except ExceptionII:
       If there is ExceptionII, then execute this block.
       ...
    else:
       If there is no exception / error then execute this block.
    finally:
       Always runs, even if an error was raised and/or if there was a `return` statement.

Note that we can still raise the same error by using the `raise` keyword in the `except` section.

We can also just check for any exception with just using except: We will look at some code that opens and writes a file:

In [11]:
try:
    f = open('testfile','w')
    f.write('Test write this')
except IOError:
    # This will only check for an IOError exception and then execute this print statement
   print("Error: Could not find file or read data")
else:
   print("Content written successfully")
   f.close()

Content written successfully


Now lets see what would happen if we did not have write permission (opening only with 'r'):

In [12]:
try:
    f = open('testfile','r')
    f.write('Test write this')
except IOError:
    # This will only check for an IOError exception and then execute this print statement
   print("Error: Could not find file or read data")
else:
   print("Content written successfully")
   f.close()

Error: Could not find file or read data


Notice how we only printed a statement! The code still ran and we were able to continue doing actions and running code blocks. This is extremely useful when you have to account for possible input errors in your code. You can be prepared for the error and keep running code, instead of your code just breaking as we saw above.

We could have also just said except: if we weren't sure what exception would occur. For example:

In [3]:
try:
    f = open('testfile','r')
    f.write('Test write this')
except:
    # This will check for any exception and then execute this print statement
   print "Error: Could not find file or read data"
else:
   print "Content written successfully"
   f.close()

Error: Could not find file or read data


We don't actually need to memorize that list of exception types! Now what if we kept wanting to run code after the exception occurred? This is where **finally** comes in.

## finally

The finally: block of code will always be run regardless if there was an exception in the try code block. The syntax is:

    try:
       Code block here
       ...
       Due to any exception, this code may be skipped!
    finally:
       This code block would always be executed.

In [4]:
try:
   f = open("testfile", "w")
   f.write("Test write statement")
finally:
   print "Always execute finally code blocks"

Always execute finally code blocks


We can use this in conjunction with except. Lets see a new example that will take into account a user putting in the wrong input:

In [5]:
def askint():
        try:
            val = int(raw_input("Please enter an integer: "))
        except:
            print "Looks like you did not enter an integer!"
            
        finally:
            print "Finally, I executed!"
        print val       

In [6]:
askint()

Please enter an integer: 3
Finally, I executed!
3


In [7]:
askint()

Please enter an integer: hai
Looks like you did not enter an integer!
Finally, I executed!


UnboundLocalError: local variable 'val' referenced before assignment

Notice how we got an error when trying to print val (because it was never properly assigned) Lets remedy this by asking the user and checking to make sure the input type is an integer:

In [8]:
def askint():
        try:
            val = int(raw_input("Please enter an integer: "))
        except:
            print "Looks like you did not enter an integer!"
            val = int(raw_input("Try again-Please enter an integer: "))
        finally:
            print "Finally, I executed!"
        print val 

In [9]:
askint()

Please enter an integer: f
Looks like you did not enter an integer!
Try again-Please enter an integer: f
Finally, I executed!


ValueError: invalid literal for int() with base 10: 'f'

That only did one check. How can we continually keep checking? We can use a while loop!

In [10]:
def askint():
    while True:
        try:
            val = int(raw_input("Please enter an integer: "))
        except:
            print "Looks like you did not enter an integer!"
            continue
        else:
            print 'Yep thats an integer!'
            break
        finally:
            print "Finally, I executed!"
        print val 

In [11]:
askint()

Please enter an integer: five
Looks like you did not enter an integer!
Finally, I executed!
Please enter an integer: ten
Looks like you did not enter an integer!
Finally, I executed!
Please enter an integer: fifteen
Looks like you did not enter an integer!
Finally, I executed!
Please enter an integer: 15
Yep thats an integer!
Finally, I executed!
